# LangChain Expression Language

In [ ]:
%pip install langchain langchain_openai langchain-community --upgrade

In [ ]:
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
)

---

## Accessing Previous Values using RunnablePassThrough

A runnable to passthrough inputs unchanged or with additional keys.

This runnable behaves almost like the identity function, except that it can be configured to add additional keys to the output, if the input is a dict.

The examples below demonstrate this runnable works using a few simple chains. The chains rely on simple lambdas to make the examples easy to execute and experiment with.

In [ ]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

print(runnable.invoke(1)) # {'origin': 1, 'modified': 2}


def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return prompt + " world"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] # Parsing logic
}

chain.invoke('hello') 

---

## Prompt + Model

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI()
prompt = ChatPromptTemplate.from_template('Tell me a joke about {topic}')

chain = prompt | chat
print(chain)

In [ ]:
print("first", chain.first)
print("last", chain.last)

In [ ]:
# Stream:
print('\n\nStream:\n')
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

# Invoke:
print('\n\nInvoke:\n')
print(chain.invoke({"topic": "bears"}).content)

# Batch:
print('\n\nBatch:\n')
print(chain.batch([{"topic": "bears"}, {"topic": "bears"}, {"topic": "bears"}]))

---

## Retrieval Augmented Generation (RAG) in LCEL

In [ ]:
%pip install langchain openai faiss-cpu tiktoken --upgrade --quiet

In [ ]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores.faiss import FAISS

In [ ]:
vectorstore = FAISS.from_texts(
    ["James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData", "James has an age of 31 years old."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# It's the same as this, but the tuple allows for line breaks:
# {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

In [ ]:
chain.invoke("What company does James phoenix work at?")

In [ ]:
chain.invoke("What is James Phoenix's age?")

---

## Understanding How `itemgetter` Works with Piping

In [ ]:
test = {
    "data": ['This is a test', 'Another entry...']
}

print(itemgetter(test))
print(itemgetter('data')(test))

### How does it work within the context of LCEL?

In [ ]:
prompt = ChatPromptTemplate.from_template('''What is the profession of James Phoenix? His profession is {profession}.''')

first_chain = RunnableParallel(
    name=lambda x: "James Phoenix",
    age=lambda x: 31
)

second_chain = {
    # itemgetter is used to get the value from the dictionary from the previous step: (note this is only the previous step, not the whole chain)
    'name': itemgetter('name'),
    'age': itemgetter('age'),
    # You can not use string values, either use itemgetter or a lambda, or RunnablePassthrough
    'profession': lambda x: "Data Engineer"
}

chain = first_chain | second_chain |  prompt |  ChatOpenAI() | StrOutputParser()
chain.invoke({})